In [1]:
import pickle
from model.Season import Season

In [4]:
with open('seasons.pickle') as f:
    seasons = pickle.load(f)

In [10]:
for year, season in seasons.iteritems():
    print year
    print '\\begin{itemize}'
    for event in season.events:
        print '  \item \\href{' + event.url + '}'
    print '\\end{itemize}\n'

2006
\begin{itemize}
  \item \href{http://www.isuresults.com/results/gpusa05/}
  \item \href{http://www.isuresults.com/results/gpcan05/}
  \item \href{http://www.isuresults.com/results/gpchn05/}
  \item \href{http://www.isuresults.com/results/gpfra05/}
  \item \href{http://www.isuresults.com/results/gprus05/}
  \item \href{http://www.isuresults.com/results/gpjpn05/}
  \item \href{http://www.isuresults.com/results/gpf0506/}
  \item \href{http://www.isuresults.com/results/ec2006/}
  \item \href{http://www.isuresults.com/results/fc2006/}
  \item \href{http://www.isuresults.com/results/owg2006/}
  \item \href{http://www.isuresults.com/results/wc2006/}
\end{itemize}

2007
\begin{itemize}
  \item \href{http://www.isuresults.com/results/gpusa06/}
  \item \href{http://www.isuresults.com/results/gpcan06/}
  \item \href{http://www.isuresults.com/results/gpchn06/}
  \item \href{http://www.isuresults.com/results/gpfra06/}
  \item \href{http://www.isuresults.com/results/gprus06/}
  \item \href{http

In [2]:
season = Season('2018')
season.fetch_info()

In [3]:
owg = season.events[-2]

In [4]:
# owg.disciplines[3].segments[1].get_raw_csv_rows()

In [5]:
owg.parse_csvs(reparse=True)

In [6]:
season.load_and_validate_scores()

http://www.isuresults.com/results/season1718/gprus2017/
http://www.isuresults.com/results/season1718/gpcan2017/
http://www.isuresults.com/results/season1718/gpchn2017/
http://www.isuresults.com/results/season1718/gpjpn2017/
http://www.isuresults.com/results/season1718/gpfra2017/
http://www.isuresults.com/results/season1718/gpusa2017/
http://www.isuresults.com/results/season1718/gpf1718/
http://www.isuresults.com/results/season1718/ec2018/
http://www.isuresults.com/results/season1718/fc2018/
http://www.isuresults.com/results/season1718/owg2018/
http://www.isuresults.com/results/season1718/wc2018/


In [11]:
season.events[-1].disciplines[0].discipline.name

'men'

In [27]:
import json
with open('pd_data/name_fixes_all.json') as f:
    name_fixes = json.load(f)
def remove_mr_ms(judge):
    judge = judge.replace('Mr. ', '')
    judge = judge.replace('Mr ', '')
    judge = judge.replace('Ms. ', '')
    judge = judge.replace('Ms ', '')
    judge = judge.replace('Mrs. ', '')
    return judge

def fix_ascii(name):
    if name in name_fixes:
        return name_fixes[name]
    if 'Jussiville PARTANEN' in name:
        return 'Cecilia TORN / Jussiville PARTANEN'
    if 'Helery' in name:
        return 'Helery HALVIN'
    if name.startswith('Victoria MANNI'):
        return 'Victoria MANNI / Carlo ROTHLISBERGER'
    if name.startswith('Anita'):
        return 'Anita OSTLUND'
    return name

def calculate_trimmed_mean(scores):
    num_scores = len(scores)
    return (sum(scores) - min(scores) - max(scores)) / (num_scores - 2.)
def is_gpf(segment_name):
    return 1 if segment_name.startswith('gpf') and not segment_name.startswith('gpfra') else 0
def is_ecfc(segment_name):
    return 1 if segment_name.startswith('ec') or segment_name.startswith('fc') else 0
def is_wc(segment_name):
    return 1 if segment_name.startswith('wc') else 0
def is_owg(segment_name):
    return 1 if segment_name.startswith('owg') else 0

In [26]:
with open('pd_data/judge_nations.json') as f:
    judge_nations = json.load(f)

In [38]:
import numpy as np
judge_scores_worlds = []
for discipline in season.events[-1].disciplines:
    discipline.create_results()
    results = {}
    for rank, result in enumerate(discipline.results):    
        skater = name_fixes.get(result.skater.name, result.skater.name)
        results[skater] = rank + 1
    num_entries = len(results)
    for segment in discipline.segments:
        for scorecard in segment.scorecards:
            skater = name_fixes.get(scorecard.skater.name, scorecard.skater.name)
            judges = [name_fixes.get(remove_mr_ms(judge.name), remove_mr_ms(judge.name))
                    for judge in segment.panel.judges if judge.name != '-']
            for element in scorecard.elements:
                med_goe = np.median(element.parsed_goes)
                trimmed_mean = calculate_trimmed_mean(element.parsed_goes)
                goes_str = ','.join(map(str, element.parsed_goes))
                for i, judge in enumerate(judges):
                    judge_num = i+1
                    judge_scores_worlds.append({
                        'skater': skater,
                        'country': scorecard.skater.country,
                        'segment_name': 'wc2018' + '_' + segment.name,
                        'season': season.champ_year,
                        'discipline': discipline.discipline.name,
                        'segment_rank': scorecard.rank,
                        'overall_rank': results[skater],
                        'num_entries': num_entries,
                        'start_order': scorecard.starting_number,
                        'elt_number': element.number,
                        'elt_name': element.name,
                        'elt_info': element.info,
                        'elt_bv': element.base_value,
                        'elt_bonus': element.bonus,
                        'elt_score': element.goe,
                        'elt_points': element.points,
                        'judge': judge,
                        'judge_num': judge_num,
                        'judge_score': element.parsed_goes[i],
                        'med_score': med_goe,
                        'trimmed_mean': trimmed_mean,
                        'all_scores': goes_str,
                        'is_comp': 0,
                        'judge_country': judge_nations[judge],
                        'same_country': 1 if judge_nations[judge] == scorecard.skater.country else 0,
                        'judge_skater_country': judge_nations[judge] + scorecard.skater.country,
                        'is_gpf': 0,
                        'is_ecfc': 0,
                        'is_wc': 1,
                        'is_owg': 0,
                    })
                for comp in scorecard.components:
                    med_score = np.median(comp.parsed_scores)
                    trimmed_mean = calculate_trimmed_mean(comp.parsed_scores)
                    scores_str = ','.join(map(str, comp.parsed_scores))
                    for i, judge in enumerate(judges):
                        judge_num = i+1
                        judge_scores_worlds.append({
                            'skater': skater,
                            'country': scorecard.skater.country,
                            'segment_name': 'wc2018' + '_' + segment.name,
                            'season': season.champ_year,
                            'discipline': discipline.discipline.name,
                            'segment_rank': scorecard.rank,
                            'overall_rank': results[skater],
                            'num_entries': num_entries,
                            'start_order': scorecard.starting_number,
                            'elt_name': name_fixes.get(comp.name, comp.name),
                            'elt_score': comp.points,
                            'judge': judge,
                            'judge_num': judge_num,
                            'judge_score': comp.parsed_scores[i],
                            'med_score': med_score,
                            'trimmed_mean': trimmed_mean,
                            'all_scores': scores_str,
                            'is_comp': 1,
                            'judge_country': judge_nations[judge],
                            'same_country': 1 if judge_nations[judge] == scorecard.skater.country else 0,
                            'judge_skater_country': judge_nations[judge] + scorecard.skater.country,
                            'is_gpf': 0,
                            'is_ecfc': 0,
                            'is_wc': 1,
                            'is_owg': 0,
                        })

In [39]:
import pandas as pd

In [40]:
judge_scores_worlds = pd.DataFrame(judge_scores_worlds)

In [41]:
judge_scores_worlds.to_csv('pd_data/judge_scores_wc2018.csv')

In [4]:
for event in season.events:
    event.write_results()

In [8]:
with open('season2018.pickle', 'w') as f:
    pickle.dump(season, f)

In [13]:
season.event_dict['gpf1718'].disciplines[1].segments[1].elt_re.match('5 2A+3T 8.36 X -0.40 -1 -1 0 1 -1 -1 0 0 -1  7.96').groups()[4] in ('x', 'X')

True

In [9]:
season.event_dict['gpf1718'].disciplines[1].segments[1].get_raw_csv_rows()

['Grand Prix Final 2017 Senior and Junior  ',
 'LADIES FREE SKATINGJUDGES DETAILS PER SKATER  ',
 'Total Total Total  ',
 'Rank Name Nation Starting Segment Element Program  Deductions',
 'Number Score Score Component  ',
 '= + Score (factorized)  -',
 '1 Alina ZAGITOVA RUS 5 147.03 76.61 70.42  0.00',
 '# Executed Info Base GOE J1 J2 J3 J4 J5 J6 J7 J8 J9  Ref Scores of',
 'Elements Value  Panel',
 '1 ChSq1 2.00 1.60 2 3 3 3 1 2 2 2 2  3.60',
 '2 FCSp4 3.20 1.14 2 2 2 3 2 3 2 3 2  4.34',
 '3 StSq4 3.90 1.40 2 2 2 2 1 2 2 2 3  5.30',
 '4 3Lz+3Lo 12.21 X 1.30 2 1 2 2 1 2 2 2 2  13.51',
 '5 2A+3T 8.36 X -0.40 -1 -1 0 1 -1 -1 0 0 -1  7.96',
 '6 3F+2T+2Lo 9.24 X 1.00 2 1 1 2 2 2 1 1 1  10.24',
 '7 LSp4 2.70 1.00 2 2 2 2 2 2 2 2 2  3.70',
 '8 3Lz 6.60 X -0.80 -2 -1 -1 0 -2 -2 0 0 -2  5.80',
 '9 3S 4.84 X 1.10 1 1 1 2 2 2 2 2 1  5.94',
 '10 3F 5.83 X 1.40 2 2 2 2 2 2 2 2 2  7.23',
 '11 2A 3.63 X 0.57 1 1 1 2 1 1 2 1 1  4.20',
 '12 CCoSp4 3.50 1.29 3 3 2 3 2 2 2 3 3  4.79',
 '66.01  76.61',
 '

In [12]:
import requests
from bs4 import BeautifulSoup

In [14]:
page = requests.get('http://www.isuresults.com/results/season1718/gpjpn2017/data0130.htm')
soup = BeautifulSoup(page.content, 'html.parser')

In [36]:
contents = [elt for elt in soup.find_all(attrs={'class': True}, name='tr')[1].contents if elt != '\n']

In [40]:
contents

[<td class="Left" nowrap="">Technical Controller</td>,
 <td align="left" nowrap="">\xa0Ms.\xa0Emilie\xa0BILLOW\xa0</td>,
 <td nowrap=""><img src="./flags/isu.gif"/>\xa0ISU</td>]

In [42]:
for tr in soup.find_all(attrs={'class': True}, name='tr'):
    print tr, tr.contents
    contents = [elt for elt in tr.contents if elt != '\n']
    print contents
    if len(contents) != 3 or contents[0].name != 'td':
        print
        continue
    for thing in contents:
        if thing.text.strip() == 'Function':
            break
        print thing.text, 'hi'
    print

<tr class="HeaderBlue"><td>Function</td><td>Name</td><td>Nation</td></tr> [<td>Function</td>, <td>Name</td>, <td>Nation</td>]
[<td>Function</td>, <td>Name</td>, <td>Nation</td>]

<tr class="Blue0">
<td class="Left" nowrap="">Technical Controller</td>
<td align="left" nowrap=""> Ms. Emilie BILLOW </td>
<td nowrap=""><img src="./flags/isu.gif"/> ISU</td></tr> [u'\n', <td class="Left" nowrap="">Technical Controller</td>, u'\n', <td align="left" nowrap="">\xa0Ms.\xa0Emilie\xa0BILLOW\xa0</td>, u'\n', <td nowrap=""><img src="./flags/isu.gif"/>\xa0ISU</td>]
[<td class="Left" nowrap="">Technical Controller</td>, <td align="left" nowrap="">\xa0Ms.\xa0Emilie\xa0BILLOW\xa0</td>, <td nowrap=""><img src="./flags/isu.gif"/>\xa0ISU</td>]
Technical Controller hi
 Ms. Emilie BILLOW hi
 ISU hi

<tr class="Blue1">
<td class="Left" nowrap="">Technical Specialist</td>
<td align="left" nowrap=""> Mr. Kelly CRUICKSHANK </td>
<td nowrap=""><img src="./flags/isu.gif"/> ISU</td></tr> [u'\n', <td class="Left" no

In [ ]:
spuo